In [4]:
%load_ext autoreload
%autoreload 2

import sys
import os
import dotenv
from pathlib import Path

env_file = "../.env"

if os.path.exists(env_file):
    dotenv.load_dotenv(env_file, verbose=True)
    print("Loaded environment variables from .env file.")

cwd = os.getcwd()
# for some reason appending to PATH you need it to be string
sys.path.append(str(Path(cwd).parent / "src"))
hf_access_token = os.getenv("HUGGINGFACE_API_KEY")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Loaded environment variables from .env file.


In [5]:
from research_tools.gpu import get_gpus_available

os.environ["CUDA_VISIBLE_DEVICES"] = ",".join([str(i) for i in get_gpus_available()])

In [6]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
assert device == torch.device("cuda")

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
updated_model = AutoModelForCausalLM.from_pretrained(
    model_id, revision="main", torch_dtype=torch.bfloat16
).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_id)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1")
updated_model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-v0.1")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
from datasets import load_dataset
import json

unlearning_task = "cyber"

forget_corpus = (
    "cyber-forget-corpus" if unlearning_task == "cyber" else "bio-forget-corpus"
)
retain_corpus = "wikitext"


def get_unlearning_rmu_dataset(name, min_len=0, batch_size=4):
    data = []
    if name == "wikitext":
        raw_data = load_dataset("wikitext", "wikitext-2-raw-v1", split="test")
        for x in raw_data:
            if len(x["text"]) > min_len:
                data.append(str(x["text"]))
    elif name == "cyber-forget-corpus":
        raw_data = load_dataset("cais/wmdp-corpora", name=name, split="train")
        for x in raw_data:
            if len(x["text"]) > min_len:
                data.append(str(x["text"]))
    else:  # wmdp bio must be pre-downloaded
        for line in open(f"data/{name}.jsonl", "r"):
            raw_text = json.loads(line)["text"]
            if len(raw_text) > min_len:
                data.append(str(raw_text))
    data = [data[i : i + batch_size] for i in range(0, len(data), batch_size)]
    return data


forget_data_list = get_unlearning_rmu_dataset(forget_corpus)
retain_data_list = get_unlearning_rmu_dataset(retain_corpus)

In [ ]:
idx = 0
max_length = 512
tokenizer.pad_token_id = tokenizer.eos_token_id

unlearn_batch = forget_data_list[idx]
retain_batch = retain_data_list[idx]

unlearn_inputs = tokenizer(
    unlearn_batch,
    return_tensors="pt",
    padding="max_length",
    truncation=True,
    max_length=max_length,
).to(updated_model.device)
retain_inputs = tokenizer(
    retain_batch,
    return_tensors="pt",
    padding="max_length",
    truncation=True,
    max_length=max_length,
).to(updated_model.device)

adv_labels_mask = torch.zeros_like(unlearn_inputs["input_ids"], dtype=bool)
def_labels_mask = torch.zeros_like(retain_inputs["input_ids"], dtype=bool)
for b, example in enumerate(retain_batch):
    len_example = len(tokenizer(example)["input_ids"])
    def_labels_mask[b, :len_example] = True
for b, example in enumerate(unlearn_batch):
    len_example = len(tokenizer(example)["input_ids"])
    adv_labels_mask[b, :len_example] = True

# prompt_mask = torch.zeros(len(unlearn_batch), pad_length + 1, dtype=torch.bool)
pgd_batch = {
    "def_tokens": retain_inputs["input_ids"].to(updated_model.device),
    "adv_tokens": unlearn_inputs["input_ids"].to(updated_model.device),
    "adv_labels_mask": adv_labels_mask.to(updated_model.device),
    "def_labels_mask": def_labels_mask.to(updated_model.device),
}

In [6]:
epsilon = 20.0
lr_adv = 1e-3
noise_completions = True
n_pgd_steps = 20

In [ ]:
from unlearn_order.algos.lat.model import LATModel


lat_model = LATModel(
    model=updated_model, tokenizer=tokenizer, layer=12, attack_completions=False
)

In [ ]:
for param in updated_model.model.layers[0].parameters():
    print()

AttributeError: 'Parameter' object has no attribute 'named_modules'

[('mlp.down_proj', Linear(in_features=14336, out_features=4096, bias=False))]

In [ ]:
from tqdm import tqdm
from unlearn_order.algos.lat.hooks import add_hooks
from unlearn_order.algos.lat.model import get_adversary_hook, projected_gradient_descent
from unlearn_order.algos.lat.utils import get_layer_module, set_model_grads
from typing import List, Dict
from transformers import LlamaForCausalLM, LlamaTokenizer


def get_layers_params(
    model: LlamaForCausalLM, layers: List[int], target_modules: List[str]
):
    params = []
    for layer in layers:
        for name, module in model.model[layer].named_modules():
            if any([target in name for target in target_modules]):
                for param in module.parameters():
                    params.append(param)
    return params


def forward_with_cache(
    model: LlamaForCausalLM,
    inputs: Dict[str, torch.Tensor],
    module: torch.nn.Module,
    no_grad: bool = True,
):
    # define a tensor with the size of our cached activations
    cache = []

    def hook(module, input, output):
        if isinstance(output, tuple):
            cache.append(output[0])
        else:
            cache.append(output)
        return None

    hook_handle = module.register_forward_hook(hook)

    if no_grad:
        with torch.no_grad():
            _ = model(**inputs)
    else:
        _ = model(**inputs)

    hook_handle.remove()

    return cache[0]


# you're optimizing layer_ids
# there is a layer where you're examining the activations at and trying to make garbage
# then you only train on the laeyrs before that, this is just shoving garbage in
# not real unlearning?
# let us also latently perturb at the same layer

from peft import PeftConfig

PeftConfig()


# remember params
def run_rmu(
    updated_model: LlamaForCausalLM,
    frozen_model: LlamaForCausalLM,
    tokenizer: LlamaTokenizer,
    forget_data_list,
    retain_data_list,
    target_modules: List[str],
    alpha: float = 1200.0,
    layer: int = 8,  # layers to do RMU in
    lr_def: float = 5.0e-5,
    max_num_batches=200,
    steering_coef: float = 6.5,
    use_lat: bool = True,
    epsilon: float = 2,
    lr_adv: float = 5e-2,
    n_pgd_steps: int = 16,
    loss_coefs: Dict[str, float] = {"toward": 1, "away": 1},
    num_epochs: int = 1,
):
    adv_layer = layer - 1
    def_layers = [layer, layer - 1, layer - 2]

    updated_model.train()

    params = get_layers_params(updated_model, def_layers, target_modules)
    optimizer = torch.optim.AdamW(params, lr=lr_def)

    updated_module = get_layer_module(updated_model, layer)
    frozen_module = get_layer_module(frozen_model, layer)

    control_vectors_list = []
    for i in range(len(forget_data_list)):
        random_vector = torch.rand(
            1,
            1,
            updated_model.config.hidden_size,
            dtype=updated_model.dtype,
            device=updated_model.device,
        )
        control_vec = random_vector / torch.norm(random_vector) * steering_coef
        control_vectors_list.append(control_vec)

    num_batches = max_num_batches
    truncation_side = tokenizer.truncation_side
    tokenizer.truncation_side = "right"

    for epoch in range(num_epochs):
        for idx in tqdm(range(num_batches)):
            control_vec = control_vectors_list[idx]
            unlearn_batch = forget_data_list[idx]
            retain_batch = retain_data_list[idx]

            max_length = 512 if idx == 0 else 768

            unlearn_inputs = tokenizer(
                unlearn_batch,
                return_tensors="pt",
                padding="max_length",
                truncation=True,
                max_length=max_length,
            ).to(updated_model.device)

            retain_inputs = tokenizer(
                retain_batch,
                return_tensors="pt",
                padding="max_length",
                truncation=True,
                max_length=max_length,
            ).to(updated_model.device)

            if use_lat:
                adv_labels_mask = torch.zeros_like(
                    unlearn_inputs["input_ids"], dtype=bool
                )
                def_labels_mask = torch.zeros_like(
                    retain_inputs["input_ids"], dtype=bool
                )

                for b, example in enumerate(retain_batch):
                    len_example = len(tokenizer(example)["input_ids"])
                    def_labels_mask[b, :len_example] = True
                for b, example in enumerate(unlearn_batch):
                    len_example = len(tokenizer(example)["input_ids"])
                    adv_labels_mask[b, :len_example] = True

                # prompt_mask = torch.zeros(len(unlearn_batch), pad_length + 1, dtype=torch.bool)
                pgd_batch = {
                    "def_tokens": retain_inputs["input_ids"].to(updated_model.device),
                    "adv_tokens": unlearn_inputs["input_ids"].to(updated_model.device),
                    "adv_labels_mask": adv_labels_mask.to(updated_model.device),
                    "def_labels_mask": def_labels_mask.to(updated_model.device),
                }
                adversary = projected_gradient_descent(
                    updated_model,
                    pgd_batch,
                    layer=layer,
                    epsilon=epsilon,
                    n_pgd_steps=n_pgd_steps,
                    lr_adv=lr_adv,
                    loss_coefs=loss_coefs,
                )

                adversary.set_attack_grads(False)

                set_model_grads(updated_model, True)

                # Unlearning loss
                with add_hooks(
                    module_forward_hooks=[
                        (
                            get_layer_module(updated_model, layer),
                            get_adversary_hook(adversary),
                        )
                    ],
                    module_forward_pre_hooks=[],
                ):
                    updated_forget_activations = forward_with_cache(
                        updated_model,
                        unlearn_inputs,
                        module=updated_module,
                        no_grad=False,
                    ).to(updated_model.device)
                    unlearn_loss = torch.nn.functional.mse_loss(
                        updated_forget_activations, control_vec
                    )
            else:
                # Unlearning loss
                updated_forget_activations = forward_with_cache(
                    updated_model, unlearn_inputs, module=updated_module, no_grad=False
                ).to(updated_model.device)
                unlearn_loss = torch.nn.functional.mse_loss(
                    updated_forget_activations, control_vec
                )

            # Retain loss
            updated_retain_activations = forward_with_cache(
                updated_model, retain_inputs, module=updated_module, no_grad=False
            ).to(updated_model.device)
            frozen_retain_activations = forward_with_cache(
                frozen_model, retain_inputs, module=frozen_module, no_grad=True
            ).to(updated_model.device)

            retain_loss = torch.nn.functional.mse_loss(
                updated_retain_activations, frozen_retain_activations
            )
            retain_loss *= alpha

            # Update model
            loss = unlearn_loss + retain_loss
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

    tokenizer.truncation_side = truncation_side

    return updated_model

In [ ]:
frozen_model = AutoModelForCausalLM.from_pretrained(model_id, revision="main").to(
    device
)
updated_model = run_rmu(
    updated_model,
    frozen_model,  # function to make another version of the model for reference embeddings
    tokenizer,  # tokenizer
    forget_data_list,  # forget data
    retain_data_list,  # retain data
    alpha=1200.0,  # coef for rmu
    layer_ids=[6, 7, 8],  # layers to train
    layer=8,  # layers to do RMU in
    param_ids=[6],  # param set to do RMU in
    lr_def=5.0e-5,  # model lr
    module_str="{model_name}.model.layers[{layer_id}]",  # to access the model layers
    steering_coef=6.5,  # coef for RMU
    model_iterations_per_step=4,  # how many steps to train the model for
    max_num_batches=200,  # max num batches
    use_lat=True,  # whether to do LAT
    lat_layer=7,  # layers to attack
    epsilon=2,  # attack l2 norm bound
    lr_adv=5.0e-2,  # attack learning rate
    n_pgd_steps=16,  # number of steps to do PGD for
    loss_coefs={"toward": 1, "away": 1},  # adv loss coefs
    num_epochs=1,  # number of epochs to train for
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)
/tmp/ipykernel_679917/2115353421.py:164: UserWarning: Using a target size (torch.Size([1, 1, 4096])) that is different to the input size (torch.Size([4, 512, 4096])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  unlearn_loss = torch.nn.functional.mse_loss(
  0%|          | 1/200 [00:09<32:04,  9.67s/it]

step 0 loss: 0.095703125


/tmp/ipykernel_679917/2115353421.py:164: UserWarning: Using a target size (torch.Size([1, 1, 4096])) that is different to the input size (torch.Size([4, 768, 4096])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  unlearn_loss = torch.nn.functional.mse_loss(
  1%|          | 2/200 [00:22<39:00, 11.82s/it]

step 1 loss: 0.06591796875


...

step 197 loss: 0.06298828125


100%|█████████▉| 199/200 [45:06<00:13, 13.60s/it]

step 198 loss: 0.06103515625


100%|██████████| 200/200 [45:20<00:00, 13.60s/it]

step 199 loss: 0.060546875
